In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### (1) 데이터 가져오기

---

- 터미널에서 `~/data/data/bike-sharing-demand` 경로에 `train.csv` 데이터를 train 변수로 가져 옵니다.
- [여기](https://www.kaggle.com/c/bike-sharing-demand/data)에서 데이터를 상세히 확인할 수 있습니다.

In [2]:
!mkdir -p ~/aiffel/bike_regression/data
!cp -r ~/data/data/* ~/aiffel/bike_regression/data

cp: '/aiffel/data/data/bike-sharing-demand' and '/aiffel/aiffel/bike_regression/data/bike-sharing-demand' are the same file
cp: '/aiffel/data/data/macbook.csv' and '/aiffel/aiffel/bike_regression/data/macbook.csv' are the same file
cp: '/aiffel/data/data/macbook_test.csv' and '/aiffel/aiffel/bike_regression/data/macbook_test.csv' are the same file
cp: '/aiffel/data/data/multi_macbook.csv' and '/aiffel/aiffel/bike_regression/data/multi_macbook.csv' are the same file
cp: '/aiffel/data/data/multi_macbook_test.csv' and '/aiffel/aiffel/bike_regression/data/multi_macbook_test.csv' are the same file


In [14]:
df = pd.read_csv('~/aiffel/bike_regression/data/bike-sharing-demand/train.csv')
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


### (2) `datetime` 컬럼을 `datetime` 자료형으로 변환하고 연, 월, 일, 시, 분, 초까지 6가지 컬럼 생성하기

---

- `to_datetime` 활용 : [링크](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html#pandas.to_datetime) 참조
- pandas datetime extract year month day 키워드로 검색해 보기

In [17]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [18]:
df['Y']=df['datetime'].dt.year
df['M']=df['datetime'].dt.month
df['D']=df['datetime'].dt.day
df['h']=df['datetime'].dt.hour
df['m']=df['datetime'].dt.minute
df['s']=df['datetime'].dt.second

### (3) `year`, `month`, `day`, `hour`, `minute`, `second` 데이터 개수 시각화하기

---

- `sns.countplot` 활용해서 시각화하기
- `subplot`을 활용해서 한 번에 6개의 그래프 함께 시각화하기

### (4) `X`, `y` 컬럼 선택 및 train/test 데이터 분리

---

- `X`(feature) 에는 어떤 데이터 컬럼을 넣어야 될지 고민해 봅시다 데이터 컬럼을 추가하거나 빼면서 잘 조합하여 데이터를 넣기 바랍니다. 만일 모든 데이터 컬럼(특징)을 넣는다면 오차 값이 말도 안 되게 적게 나올 수도 있습니다. ex) count를 예측하는데 casual,registered 컬럼이 필요할까요?
    
- `count` 값을 맞추고자 하므로, `y` 변수에 `count` 컬럼의 데이터 넣기

In [32]:
# X = df.iloc[:, :-1]
X = df.drop(columns=['datetime'])
X = X.drop(columns=['count'])
y = df['count']
X

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,Y,M,D,h,m,s
0,1,0,0,1,9.84,14.395,81,0.0000,3,13,2011,1,1,0,0,0
1,1,0,0,1,9.02,13.635,80,0.0000,8,32,2011,1,1,1,0,0
2,1,0,0,1,9.02,13.635,80,0.0000,5,27,2011,1,1,2,0,0
3,1,0,0,1,9.84,14.395,75,0.0000,3,10,2011,1,1,3,0,0
4,1,0,0,1,9.84,14.395,75,0.0000,0,1,2011,1,1,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,7,329,2012,12,19,19,0,0
10882,4,0,1,1,14.76,17.425,57,15.0013,10,231,2012,12,19,20,0,0
10883,4,0,1,1,13.94,15.910,61,15.0013,4,164,2012,12,19,21,0,0
10884,4,0,1,1,13.94,17.425,61,6.0032,12,117,2012,12,19,22,0,0


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(
        X, y,
        test_size=0.2,
        random_state=42
    )

### (5) LinearRegression 모델 학습

---

- `sklearn`의 `LinearRegression` 모델 불러오기 및 학습하기

In [34]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

### (6) 학습된 모델로 `X_test`에 대한 예측값 출력 및 손실함수값 계산

---

- 학습된 모델에 `X_test`를 입력해서 예측값 출력하기
- 모델이 예측한 값과 정답 target 간의 손실함수 값 계산하기
- mse 값과 함께 rmse 값도 계산하기

In [35]:
model.fit(X_train, y_train)

LinearRegression()

In [36]:
predictions = model.predict(X_test)
predictions

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, predictions)
mse

2.3432220695614914e-25

### (7) x축은 `temp` 또는 `humidity`로, y축은 `count`로 예측 결과 시각화하기

---

- x축에 X 데이터 중 `temp` 데이터를, y축에는 count 데이터를 넣어서 시각화하기
- x축에 X 데이터 중 `humidity` 데이터를, y축에는 count 데이터를 넣어서 시각화하기

In [37]:
plt.scatter(X_test[:, 0], y_test, label="true")
plt.scatter(X_test[:, 0], predictions, label="pred")
plt.legend()
plt.show()

TypeError: '(slice(None, None, None), 0)' is an invalid key